### requirements

In [6]:
import torch

### all import tests

In [1]:
# utils
from mltools.utils import cuda_tools

# single file modules
from mltools.distributions import DiagonalGaussianDistribution
from mltools.losses import MultiScaleMSE

# networks
from mltools.networks.blocks import AttnBlock, ResNetBlock, ResNetDown, ResNetUp
from mltools.networks.network_tools import zero_init, get_conv, get_timestep_embedding
from mltools.networks.networks import CUNet, Encoder, Decoder

# models
from mltools.models.model_tools import kl_std_normal, FixedLinearSchedule, SigmoidSchedule, LearnedLinearSchedule, NNSchedule
from mltools.models.vae_model import AutoencoderKL
from mltools.models.vdm_model import VDM, LightVDM

# h5
from mltools.h5 import h5_tools

# connectomics
from mltools.connectomics import ConnectomicsDataset
from mltools.connectomics import connectomics_tools

/n/home12/cfpark00/venv1/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: rch is an invalid version and will not be supported in a future release
  warnings.warn(
/n/home12/cfpark00/venv1/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: rch is an invalid version and will not be supported in a future release
  warnings.warn(


In [4]:
from mltools.utils import cuda_tools
device=cuda_tools.get_freer_device()

memory_available [78291]
best GPU: 0


In [2]:
from mltools.models.vae_model import AutoencoderKL
enc_dec_params = dict(
    shape=(3,256,256),
    chs=[48, 96, 192],
    attn_sizes=[],
    mid_attn=False,
    num_res_blocks=1,
    dropout_prob=0.0,
    z_channels=4,
    double_z=True,
    n_attention_heads=1,
    norm_groups=8,
    norm_eps=1e-6,
    norm_affine=True,
    act="gelu",
    conv_kernel_size=3,
    conv_padding_mode="zeros",
)
vae=AutoencoderKL(enc_dec_params=enc_dec_params)
vae=vae.to(device)
x=torch.rand(1,3,256,256).to(device)
x_rec=vae.get_reconstuctions(x)
x.shape,x_rec.shape

(torch.Size([1, 3, 256, 256]), torch.Size([1, 3, 256, 256]))

In [26]:
from mltools.networks.networks import CUNet
shape=(3,256,256)
score_model=CUNet(
    shape=shape,
    chs=[48, 48, 192, 200],
    s_conditioning_channels=2,
    v_conditioning_dims = [3,6],
    v_embedding_dim = 64,
    t_conditioning=True,
    t_embedding_dim=64,
    mid_attn=True,
    dropout_prob=0.0,
    n_attention_heads=1,
    norm_groups=8,
    conv_padding_mode="zeros",
)
score_model=score_model.to(device)
batch_size=2
x=torch.randn(batch_size,*shape).to(device)
t=torch.randn(batch_size).to(device)
s=torch.randn(batch_size,2,*shape[1:]).to(device)
v=[torch.randn(batch_size,3).to(device),torch.randn(batch_size,6).to(device)]

In [27]:
res=score_model(x,t=t,s_conditioning=s,v_conditionings=v)
x.shape,res.shape

(torch.Size([2, 3, 256, 256]), torch.Size([2, 3, 256, 256]))

In [31]:
from mltools.models.vdm_model import VDM
vdm=VDM(
    score_model=score_model,
    noise_schedule= "fixed_linear",
    gamma_min = -13.3,
    gamma_max = 5.0,
    antithetic_time_sampling = True,
    image_shape = shape,
    data_noise = 1.0e-3,
)
vdm=vdm.to(device)

In [32]:
res=vdm.sample(batch_size=batch_size,conditioning=s,conditioning_values=v,n_sampling_steps=50,device=device)
res.shape

torch.Size([2, 3, 256, 256])

In [1]:
from mltools.utils import cuda_tools
device=cuda_tools.get_freer_device()
import torch

memory_available [78291]
best GPU: 0
